In [1]:
!pip install onnx tf2onnx onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.1 MB 5.8 MB/s 
     |████████████████████████████████| 442 kB 51.4 MB/s 
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0.7
    Uninstalling flatbuffers-2.0.7:
      Successfully uninstalled flatbuffers-2.0.7


In [ ]:
!unzip /content/weights.zip

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
import onnx
import tf2onnx

In [2]:
model = tf.keras.models.Sequential([
                                    Dense(24,activation='relu',input_shape = (132,)),
                                    Dense(48,activation='relu'),
                                    Dense(24,activation='relu'),
                                    Dense(4,activation='softmax')
                                   ])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                3192      
                                                                 
 dense_1 (Dense)             (None, 48)                1200      
                                                                 
 dense_2 (Dense)             (None, 24)                1176      
                                                                 
 dense_3 (Dense)             (None, 4)                 100       
                                                                 
Total params: 5,668
Trainable params: 5,668
Non-trainable params: 0
_________________________________________________________________


In [3]:
model.load_weights('/content/weights')

In [4]:
onnx_model, _ = tf2onnx.convert.from_keras(model, 
                                           [tf.TensorSpec([None, 132], dtype=tf.float32, name='x')], 
                                           opset=13)
onnx.save(onnx_model, "/model.onnx")

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


In [5]:
import onnxruntime as ort
import numpy as np

In [6]:
model = onnx.load("/model.onnx")

In [7]:
onnx.checker.check_model(model)

In [8]:
print(onnx.helper.printable_graph(model.graph))

graph tf2onnx (
  %x[FLOAT, unk__6x132]
) initializers (
  %sequential/dense_3/MatMul/ReadVariableOp:0[FLOAT, 24x4]
  %sequential/dense_3/BiasAdd/ReadVariableOp:0[FLOAT, 4]
  %sequential/dense_2/MatMul/ReadVariableOp:0[FLOAT, 48x24]
  %sequential/dense_2/BiasAdd/ReadVariableOp:0[FLOAT, 24]
  %sequential/dense_1/MatMul/ReadVariableOp:0[FLOAT, 24x48]
  %sequential/dense_1/BiasAdd/ReadVariableOp:0[FLOAT, 48]
  %sequential/dense/MatMul/ReadVariableOp:0[FLOAT, 132x24]
  %sequential/dense/BiasAdd/ReadVariableOp:0[FLOAT, 24]
) {
  %sequential/dense/MatMul:0 = MatMul(%x, %sequential/dense/MatMul/ReadVariableOp:0)
  %sequential/dense/BiasAdd:0 = Add(%sequential/dense/MatMul:0, %sequential/dense/BiasAdd/ReadVariableOp:0)
  %sequential/dense/Relu:0 = Relu(%sequential/dense/BiasAdd:0)
  %sequential/dense_1/MatMul:0 = MatMul(%sequential/dense/Relu:0, %sequential/dense_1/MatMul/ReadVariableOp:0)
  %sequential/dense_1/BiasAdd:0 = Add(%sequential/dense_1/MatMul:0, %sequential/dense_1/BiasAdd/ReadVaria

In [9]:
import onnxruntime as ort

ort_session = ort.InferenceSession("/model.onnx")

outputs = ort_session.run(
    None,
    {'x': np.random.randn(3, 132).astype(np.float32)},
)
print(outputs[0])

[[1.05855778e-23 1.00000000e+00 5.49283165e-18 6.15407362e-18]
 [9.99982119e-01 1.16344795e-14 3.22950899e-15 1.79377548e-05]
 [6.71118239e-12 9.99989867e-01 1.00482448e-05 1.05766567e-07]]
